## Submission 3: CNN

For our third submission, we designed a covolutional neural network that includes 2 hidden layers. Why we chose this configuration follows:

The first step is to import everything that we will be using

In [1]:
#imports
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


After importing, we set some hyper parameters. These hyper parameters were first set randomly, then through running the model over and over again, we found that these paramters produces a good result. While these are most likely not the optimal configuations for this network, we found that they were close and provided a good result.

In [2]:
#Set Hyper Parameters:
max_features = 5000
batch_size = 32
embedding_dims = 20
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 10
patience = 2

The next step is to import the data. We import just the training data in this step as that is what we will be using to train the model. After checking that our data looks as expected. We transform the author initals into a 0,1 or 2. This become our labels, or y of the training data. The text becomes our x.

In [3]:
# Read in the Data
train = pd.read_csv('train.csv')
print(train.head())
print("training data shape:", train.shape)

test = pd.read_csv('test.csv')
print(train.head())
print("training data shape:", train.shape)

#Transform Author and Split X and Y
a2c = {'EAP': 0, 'HPL' : 1, 'MWS' : 2}
labels = np.array([a2c[a] for a in train.author])
labels = to_categorical(labels)

print("train labels: ", labels)
text = train["text"]
print("train text shape: ", text.shape)

        id                                               text author
0  id26305  This process, however, afforded me no means of...    EAP
1  id17569  It never once occurred to me that the fumbling...    HPL
2  id11008  In his left hand was a gold snuff box, from wh...    EAP
3  id27763  How lovely is spring As we looked from Windsor...    MWS
4  id12958  Finding nothing else, not even gold, the Super...    HPL
training data shape: (19579, 3)
        id                                               text author
0  id26305  This process, however, afforded me no means of...    EAP
1  id17569  It never once occurred to me that the fumbling...    HPL
2  id11008  In his left hand was a gold snuff box, from wh...    EAP
3  id27763  How lovely is spring As we looked from Windsor...    MWS
4  id12958  Finding nothing else, not even gold, the Super...    HPL
training data shape: (19579, 3)
train labels:  [[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 1.  0.  0.]
 ..., 
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 0.  1.  

The next step is to pre process out data. Intrestingly enough, all of the pre-processing we tried made the result worse! One thing that we tried was removing stop words. This did not increase the accuracy of our model, instead it reduced it. There is a reason that we think this may have happened. Most of the text is written in an older style of English than what we use today. Removing stop words that are used in modern language is only helpful if those stopwords are present. Addititonally those stop words indicate a more causal form of writing and could have distinguished one author from another. We also tried to change the tokenizer to ignore case, remove puncuation ect. This did not increase the accuracy of our model. Finally, we pad the text so that all of the inputs are the same lenght.

In [4]:
#Pre-Processing

#removing stopwords reduces accuracy

#Tokenize
tokenizer = Tokenizer(filters="", lower=True, split=" ", char_level=False)
tokenizer.fit_on_texts(text);
docs = tokenizer.texts_to_sequences(text);
print("text: ", text[0])
print("tokenized: ", docs[0])
maxlen = np.amax([len(x) for x in docs], axis=0)
print("max doc length: ", maxlen)

#Pad to all doc are the same length
print('Pad sequences (samples x time)')
docs = sequence.pad_sequences(docs, maxlen=maxlen)
print('padded docs shape:', docs.shape)


text:  This process, however, afforded me no means of ascertaining the dimensions of my dungeon; as I might make its circuit, and return to the point whence I set out, without being aware of the fact; so perfectly uniform seemed the wall.
tokenized:  [26, 8207, 142, 1331, 31, 37, 272, 2, 12060, 1, 5088, 2, 10, 22483, 16, 6, 75, 160, 45, 22484, 3, 341, 4, 1, 274, 1996, 6, 302, 1062, 121, 123, 886, 2, 1, 22485, 39, 1332, 6237, 88, 1, 2401]
max doc length:  861
Pad sequences (samples x time)
padded docs shape: (19579, 861)


Next we split the training data into test and training data so we can see the accuracy of our model using label data.

In [5]:
#Split the testing and training data
x_train, x_test, y_train, y_test = train_test_split(docs, labels, test_size=0.2)

Finally, we are at the step to create the model. To begin we apply an Embedding layer. To prevent overtraining the model, we add a drop-out layer of 0.2. Then we add a convolutional layer with 250 filters. After that we added two hidden layers and the a softmax layer. The configuration of this model was trial and error as we tried different conovlutional layers and hidden layers. We found that the resulting configuration produces the best results.

In [6]:
#Create the model
model = Sequential()


#Embedding layer
input_dim = np.max(docs) + 1

model.add(Embedding(input_dim,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

# Convolutional Layer
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))

# Max Pooling Layer
model.add(GlobalMaxPooling1D())

# Hidden Layer
model.add(Dense(hidden_dims+200))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# Hidden Layer
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

#SoftMax Layer
model.add(Dense(3, activation='softmax'))

#Compile the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

#Train and test the model
model.fit(x_train, y_train,
                 batch_size=batch_size,
                 validation_data=(x_test, y_test),
                 epochs=epochs,
                 callbacks=[EarlyStopping(patience=patience, monitor='val_loss')])

Train on 15663 samples, validate on 3916 samples
Epoch 1/10
15663/15663 [==============================] - 10s 644us/step - loss: 0.5354 - acc: 0.7325 - val_loss: 0.3763 - val_acc: 0.8355
Epoch 2/10
15663/15663 [==============================] - 8s 513us/step - loss: 0.2608 - acc: 0.8924 - val_loss: 0.3174 - val_acc: 0.8676
Epoch 3/10
15663/15663 [==============================] - 8s 510us/step - loss: 0.1131 - acc: 0.9570 - val_loss: 0.3539 - val_acc: 0.8607
Epoch 4/10
15663/15663 [==============================] - 8s 503us/step - loss: 0.0544 - acc: 0.9808 - val_loss: 0.4197 - val_acc: 0.8560


In [7]:
print("Generating Kaggle test results - test_results.csv")
test_data = pd.read_csv("test.csv")
data = test_data["text"]

docs = tokenizer.texts_to_sequences(data);

#Pad to all doc are the same length
docs = sequence.pad_sequences(docs, maxlen=maxlen)

out = pd.DataFrame(model.predict(docs))
ids = test_data["id"]
data_to_write = pd.concat([ids, out], axis=1)
data_to_write.columns =["id", "EAP", "HPL", "MWS"]
data_to_write.to_csv("test_results.csv", index=False)

Generating Kaggle test results - test_results.csv
